<pre>
<b>
COMP-6721 | 2021-Winter
Project | Part-1

Pravesh Gupta | 40152506
Vikramjeet Singh | 
Manjot Kaur Dherdi | 
</b>
</pre>

# 1. Data Loading

## 1.1. Installing required python modules

In [1]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install torchvision

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install torchvision

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install scikit-image

In [6]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install sklearn

Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install pillow

Note: you may need to restart the kernel to use updated packages.


### 1.2. Dataset Description


Required Dataset Folder Heirarchy:
<pre>
{dataset_folder}
--images
----mask
-------{mask images}
----no_mask
-------{no mask images}
----not_person
-------{not person images}
</pre>

### 1.3. Defining Pytorch dataset Representation and data transaformers

In [1]:
from torchvision.datasets import VisionDataset
import pandas as pd
import os
from skimage import io as sk_io, transform as sk_transform
import numpy as np
from sklearn.utils import shuffle
from PIL import Image
    
class Rescale(object):
    def __init__(self, output_size, debug=False, export_path=None):
        assert isinstance(output_size, (int))
        self.output_size = output_size

    def __call__(self, img_data):
        img_arr = np.array(img_data)
        h, w, c = img_arr.shape
        isAlreadyScaled = (h==self.output_size and w==self.output_size)
        
        if not isAlreadyScaled:
            scale_factor = float(self.output_size)/img_arr.shape[0]
            img_arr = (sk_transform.rescale(img_arr, (scale_factor, scale_factor, 1))*255).astype(np.uint8)

            new_w = img_arr.shape[1]

            # Clipping or filling
            if new_w>self.output_size:
                mid = new_w//2
                new_w_start = mid-self.output_size//2
                new_w_end = mid+self.output_size//2

                if (new_w_end-new_w_start)<self.output_size:
                    new_w_end += (self.output_size-(new_w_end-new_w_start))
                elif (new_w_end-new_w_start)>self.output_size:
                    new_w_end -= ((new_w_end-new_w_start)-self.output_size)
                img_arr = img_arr[:, new_w_start:new_w_end]
            elif new_w<self.output_size:
                mid = new_w//2
                new_w_start = self.output_size//2-mid
                new_w_end = new_w_start+new_w
                filled_img_arr = np.zeros((self.output_size, self.output_size, img_arr.shape[2]), dtype=np.uint8)
                filled_img_arr[:, new_w_start:new_w_end] = img_arr[:, :]
                img_arr = filled_img_arr
        return Image.fromarray(img_arr)

### 1.4. Creating dataset and data loaders for train and test

In [2]:
def get_train_test_indices(dataset_targets):
    test_indices_map = {}
    for class_name in dataset.class_to_idx:
        label = dataset.class_to_idx[class_name]
        test_indices_map[label] = {'indices': np.array([], dtype=np.int32), 'count': 0}

    train_indices_map = {}
    for class_name in dataset.class_to_idx:
        label = dataset.class_to_idx[class_name]
        train_indices_map[label] = {'indices': np.array([], dtype=np.int32), 'count': 0}

    targets = np.array(dataset.targets, dtype=np.int32)
    target_indices = np.where(targets!=None)[0]
    np.random.shuffle(target_indices)

    for i in target_indices:
        label = dataset.targets[i]
        if test_indices_map[label]['count']<100:
            test_indices_map[label]['indices'] = np.append(test_indices_map[label]['indices'], i)
            test_indices_map[label]['count']+=1
        elif train_indices_map[label]['count']<350:
            train_indices_map[label]['indices'] = np.append(train_indices_map[label]['indices'], i)
            train_indices_map[label]['count']+=1

    test_indices = np.array([], dtype=np.int32)
    train_indices = np.array([], dtype=np.int32)

    for class_name in dataset.class_to_idx:
        label = dataset.class_to_idx[class_name]
        label_test_indices = test_indices_map[label]['indices']
        test_indices = np.append(test_indices, label_test_indices)

        label_train_indices = train_indices_map[label]['indices']
        train_indices = np.append(train_indices, label_train_indices)
    return train_indices, test_indices

In [3]:
from torchvision.transforms import ToTensor, Compose, Normalize
import torch
from torch.utils.data import DataLoader, Subset
from torchvision.datasets import ImageFolder
import gc

torch.manual_seed(6721)

rescaled_size=512

data_dir = './data/'

data_transform = Compose([
    Rescale(rescaled_size)
    , ToTensor()
])

dataset = ImageFolder(
    root=data_dir
    ,transform=data_transform
)
print(f'Dataset: size: {len(dataset)}, class labels: {dataset.class_to_idx}')

train_indices, test_indices = get_train_test_indices(dataset.targets)
selected_indices = np.concatenate((train_indices, test_indices))

selected_dataset = Subset(dataset, selected_indices)
print(f'Selected dataset: size: {len(selected_dataset)}')
selected_dataloader = DataLoader(selected_dataset, batch_size=8, shuffle=False)

means = torch.tensor([])
stds = torch.tensor([])
for i, (data, labels) in enumerate(selected_dataloader):
    gc.collect()
    batch_mean = torch.mean(data, axis=(0, 2, 3))
    batch_std = torch.std(data, axis=(0, 2, 3))
    means = torch.cat((means, batch_mean.unsqueeze(0)))
    stds = torch.cat((stds, batch_std.unsqueeze(0)))
mean = torch.mean(means, axis=0)
std = torch.mean(stds, axis=0)
print(f'Mean: {mean}')
print(f'Std: {std}')

Dataset: size: 1344, class labels: {'mask': 0, 'no_mask': 1, 'not_person': 2}
Selected dataset: size: 1344
Mean: tensor([0.4322, 0.4013, 0.3816])
Std: tensor([0.2814, 0.2715, 0.2818])


In [8]:
torch.manual_seed(6721)

from torchvision.transforms import ToTensor, Compose, Normalize
import torch
from torch.utils.data import DataLoader, Subset
from torchvision.datasets import ImageFolder

data_transform = Compose([
    Rescale(rescaled_size)
    , ToTensor()
    , Normalize(mean=mean, std=std)
])

dataset = ImageFolder(
    root=data_dir
    ,transform=data_transform
)

train_dataset = Subset(dataset, train_indices)
print(f'Train dataset: size: {len(train_dataset)}')

train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
for data, labels in train_dataloader:
    print(f'Train X | y batch shapes : {data.shape, labels.shape}')
    break

test_dataset = Subset(dataset, test_indices)
print(f'Test dataset: size: {len(test_dataset)}')

test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=True)
for data, labels in test_dataloader:
    print(f'Test X | y batch shapes : {data.shape, labels.shape}')
    break

Train dataset: size: 1044
Train X | y batch shapes : (torch.Size([8, 3, 512, 512]), torch.Size([8]))
Test dataset: size: 300
Test X | y batch shapes : (torch.Size([8, 3, 512, 512]), torch.Size([8]))


# 2. Data Modelling

In [9]:
from torch.nn import Module, Conv2d, MaxPool2d, Linear, ReLU

class ProjectModel(Module):
    def __init__(self, num_classes=3):
        super(ProjectModel, self).__init__()
        self.module = torch.nn.Sequential(
            torch.nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=2)
            , torch.nn.BatchNorm2d(32)
            , torch.nn.LeakyReLU(inplace=True)
            
            , torch.nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=0)
            , torch.nn.BatchNorm2d(32)
            , torch.nn.LeakyReLU(inplace=True)
            
            , torch.nn.MaxPool2d(kernel_size=2, stride=2)
            
            , torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=0)
            , torch.nn.BatchNorm2d(64)
            , torch.nn.LeakyReLU(inplace=True)
            
            , torch.nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=0)
            , torch.nn.BatchNorm2d(64)
            , torch.nn.LeakyReLU(inplace=True)
            
            , torch.nn.MaxPool2d(kernel_size=2, stride=2)
            
            , torch.nn.Flatten()
            , torch.nn.Linear(64*126*126, num_classes)
        )
        
    def forward(self, X):
        return self.module(X)

### 2.1. Integrity Test

In [10]:
torch.manual_seed(6721)

from sklearn import metrics

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

net = ProjectModel()

# net = net.to(device)

# Model Unit test
total_labels = 0
correct_labels = 0
for data, labels in train_dataloader:
#     torch.cuda.empty_cache()
#     data, labels = data.to(device), labels.to(device)
    outputs = net(data)
    print(outputs)
    y_pred = torch.argmax(outputs, dim=1)
    print(y_pred)
    total_labels+=labels.size(0)
    correct_labels += (y_pred==labels).sum().item()
    break

print(f'Accuracy: {(correct_labels/total_labels)*100}')
print('Model test passed.')

tensor([[ 0.8427,  0.9975,  0.7415],
        [ 2.3258,  0.4306,  0.6336],
        [ 0.9899,  0.0332,  0.4025],
        [ 0.1196,  1.0243, -0.4477],
        [ 0.5997,  0.5001,  0.4291],
        [ 1.0980,  0.6828,  0.0372],
        [ 1.3799,  1.5422, -0.3335],
        [ 0.9429,  0.4780,  0.2456]], grad_fn=<AddmmBackward>)
tensor([1, 0, 0, 1, 0, 0, 1, 0])
Accuracy: 50.0
Model test passed.


### 2.2. Training

In [11]:
torch.manual_seed(6721)

from torch.optim import SGD
from torch.nn import CrossEntropyLoss
import time

num_epoch = 5
lr = 0.001
momentum = 0.5
net = ProjectModel()

loss_evaluater = CrossEntropyLoss()
optimizer = SGD(net.parameters(), lr=lr, momentum=momentum)

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
# net = net.to(device)

net.train()

t = time.time()
for i in range(num_epoch):
    print(f'Epoch: {i+1}')
    t_epoch = time.time()
    total_labels = 0
    correctly_pred = 0
    t_batch = time.time()
    for j, (data, labels) in enumerate(train_dataloader):
#         torch.cuda.empty_cache()
        gc.collect()
#         data, labels = data.to(device), labels.to(device)
        X, y = data, labels
        optimizer.zero_grad()
        output = net(X)
        loss = loss_evaluater(output, y)
        loss.backward()
        optimizer.step()
        y_pred = torch.argmax(output, dim=1)
        total_labels += y.size(0)
        correctly_pred += (y_pred==y).sum().item()
        current_acc = (correctly_pred/total_labels)
        if j%5==0:
            seconds_passed = time.time()-t_batch
            print(f'After {j} batches: [time:{seconds_passed//60}m {seconds_passed%60}s, Accuracy: {current_acc*100}]')
    epoch_acc = (correctly_pred/total_labels)
    seconds_passed = time.time()-t_epoch
    print(f'Epoch {i+1}: [time:{seconds_passed//60}m {seconds_passed%60}s, Accuracy: {epoch_acc*100}]')
    print()
seconds_passed = time.time()-t
print(f'Training time: {seconds_passed//60}m {seconds_passed%60}s')

Epoch: 1
After 0 batches: [time:0.0m 7.9788360595703125s, Accuracy: 50.0]
After 5 batches: [time:0.0m 37.384806632995605s, Accuracy: 37.5]
After 10 batches: [time:1.0m 5.338175296783447s, Accuracy: 36.36363636363637]
After 15 batches: [time:1.0m 33.06679153442383s, Accuracy: 36.71875]
After 20 batches: [time:2.0m 0.7114949226379395s, Accuracy: 39.285714285714285]
After 25 batches: [time:2.0m 28.31422758102417s, Accuracy: 40.86538461538461]
After 30 batches: [time:2.0m 58.5833740234375s, Accuracy: 39.516129032258064]
After 35 batches: [time:3.0m 26.927509546279907s, Accuracy: 39.58333333333333]
After 40 batches: [time:3.0m 56.62056827545166s, Accuracy: 39.02439024390244]
After 45 batches: [time:4.0m 25.321579933166504s, Accuracy: 40.21739130434783]
After 50 batches: [time:4.0m 53.31445503234863s, Accuracy: 41.17647058823529]
After 55 batches: [time:5.0m 21.514552116394043s, Accuracy: 41.294642857142854]
After 60 batches: [time:5.0m 49.809208393096924s, Accuracy: 40.16393442622951]
After

After 110 batches: [time:11.0m 1.7149364948272705s, Accuracy: 77.92792792792793]
After 115 batches: [time:11.0m 30.767980098724365s, Accuracy: 77.69396551724138]
After 120 batches: [time:11.0m 59.5701322555542s, Accuracy: 78.099173553719]
After 125 batches: [time:12.0m 30.950241804122925s, Accuracy: 77.97619047619048]
After 130 batches: [time:12.0m 57.68120050430298s, Accuracy: 78.06513409961686]
Epoch 4: [time:12.0m 57.68120050430298s, Accuracy: 78.06513409961686]

Epoch: 5
After 0 batches: [time:0.0m 5.480627775192261s, Accuracy: 87.5]
After 5 batches: [time:0.0m 34.84145927429199s, Accuracy: 83.33333333333334]
After 10 batches: [time:1.0m 5.627687692642212s, Accuracy: 86.36363636363636]
After 15 batches: [time:1.0m 37.69657325744629s, Accuracy: 85.15625]
After 20 batches: [time:2.0m 7.762336015701294s, Accuracy: 86.30952380952381]
After 25 batches: [time:2.0m 37.75038409233093s, Accuracy: 86.53846153846155]
After 30 batches: [time:3.0m 6.924262285232544s, Accuracy: 86.69354838709677

### 2.3. Training Evaluation

In [14]:
torch.manual_seed(6721)

t = time.time()
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')

with torch.no_grad():
    total_labels = 0
    correctly_pred = 0
    t_batch = time.time()
    for i, (data, labels) in enumerate(train_dataloader):
#         data,labels = data.to(device), labels.to(device)
        gc.collect()
        X, y = data, labels
        output = net(X)
        y_pred = torch.argmax(output, dim=1)
        total_labels += y.size(0)
        correctly_pred += (y_pred==y).sum().item()
        current_acc = (correctly_pred/total_labels)
        if i%5==0:
            seconds_passed = time.time()-t_batch
            print(f'After {i} batches: [time:{seconds_passed//60}m {seconds_passed%60}s, Accuracy: {current_acc*100}]')
    train_acc = (correctly_pred/total_labels)
    print(f'Training accuracy: {train_acc*100}')
seconds_passed = time.time()-t

print(f'Trainig evaluation time: {seconds_passed//60}m {seconds_passed%60}s')

After 0 batches: [time:0.0m 2.034867286682129s, Accuracy: 100.0]
After 5 batches: [time:0.0m 12.793136358261108s, Accuracy: 97.91666666666666]
After 10 batches: [time:0.0m 23.726908206939697s, Accuracy: 94.31818181818183]
After 15 batches: [time:0.0m 34.384806871414185s, Accuracy: 90.625]
After 20 batches: [time:0.0m 44.63647174835205s, Accuracy: 91.07142857142857]
After 25 batches: [time:0.0m 54.9376962184906s, Accuracy: 92.3076923076923]
After 30 batches: [time:1.0m 5.849656581878662s, Accuracy: 91.53225806451613]
After 35 batches: [time:1.0m 16.235935926437378s, Accuracy: 91.31944444444444]
After 40 batches: [time:1.0m 27.290453910827637s, Accuracy: 91.46341463414635]
After 45 batches: [time:1.0m 38.015302896499634s, Accuracy: 91.30434782608695]
After 50 batches: [time:1.0m 49.54110860824585s, Accuracy: 90.93137254901961]
After 55 batches: [time:2.0m 0.4509751796722412s, Accuracy: 91.29464285714286]
After 60 batches: [time:2.0m 11.839511156082153s, Accuracy: 90.98360655737704]
After

### 2.4. Testing

In [15]:
torch.manual_seed(6721)

net.eval()

t = time.time()
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')

total_labels = 0
correctly_pred = 0
t_batch = time.time()
for i, (data, labels) in enumerate(test_dataloader):
#     torch.cuda.empty_cache()
#     data, labels = data.to(device), labels.to(device)
    gc.collect()
    X, y = data, labels
    output = net(X)
    y_pred = torch.argmax(output, dim=1)
    total_labels += y.size(0)
    correctly_pred += (y_pred==y).sum().item()
    current_acc = (correctly_pred/total_labels)
    if i%5==0:
        seconds_passed = time.time()-t_batch
        print(f'After {i} batches: [time:{seconds_passed//60}m {seconds_passed%60}s, Accuracy: {current_acc*100}]')

test_acc = (correctly_pred/total_labels)
print(f'Test accuracy: {test_acc*100}')
seconds_passed = time.time()-t
print(f'Test time: {seconds_passed//60}m {seconds_passed%60}s')

After 0 batches: [time:0.0m 1.9298062324523926s, Accuracy: 50.0]
After 5 batches: [time:0.0m 17.20026469230652s, Accuracy: 72.91666666666666]
After 10 batches: [time:0.0m 28.88688635826111s, Accuracy: 75.0]
After 15 batches: [time:0.0m 40.97071886062622s, Accuracy: 75.0]
After 20 batches: [time:0.0m 52.90430927276611s, Accuracy: 72.61904761904762]
After 25 batches: [time:1.0m 4.585077285766602s, Accuracy: 69.71153846153845]
After 30 batches: [time:1.0m 16.374597549438477s, Accuracy: 68.95161290322581]
After 35 batches: [time:1.0m 28.56737470626831s, Accuracy: 68.40277777777779]
Test accuracy: 67.66666666666666
Test time: 1.0m 32.29839634895325s


### 2.5. Saving pretrained Model

In [19]:
torch.manual_seed(6721)

file_path = './net.pt'

state = {
    'epoch': num_epoch,
    'state_dict': net.state_dict(),
    'optimizer': optimizer.state_dict(),
    'lr': lr,
    'momentum': momentum
}

torch.save(state, file_path)

### 2.6. Loading Pretrained Model

In [20]:
torch.manual_seed(6721)

state = torch.load(file_path)

loaded_net = ProjectModel()
loaded_loss_evaluater = CrossEntropyLoss()
loaded_optimizer = SGD(net.parameters(), lr=lr, momentum=momentum)

loaded_num_epoch = state['epoch']
loaded_lr = state['lr']
loaded_momentum = state['momentum']
loaded_net.load_state_dict(state['state_dict'])
loaded_optimizer.load_state_dict(state['optimizer'])

Testing loaded pretrained Model

In [21]:
torch.manual_seed(6721)

loaded_net.eval()

t = time.time()

total_labels = 0
correctly_pred = 0
t_batch = time.time()
for i, (data, labels) in enumerate(test_dataloader):
#     torch.cuda.empty_cache()
#     data, labels = data.to(device), labels.to(device)
    gc.collect()
    X, y = data, labels
    output = loaded_net(X)
    y_pred = torch.argmax(output, dim=1)
    total_labels += y.size(0)
    correctly_pred += (y_pred==y).sum().item()
    current_acc = (correctly_pred/total_labels)
    if i%5==0:
        seconds_passed = time.time()-t_batch
        print(f'After {i} batches: [time:{seconds_passed//60}m {seconds_passed%60}s, Accuracy: {current_acc*100}]')

test_acc = (correctly_pred/total_labels)
print(f'Test accuracy: {test_acc*100}')
seconds_passed = time.time()-t
print(f'Test time: {seconds_passed//60}m {seconds_passed%60}s')

After 0 batches: [time:0.0m 2.1682002544403076s, Accuracy: 50.0]
After 5 batches: [time:0.0m 13.696052074432373s, Accuracy: 72.91666666666666]
After 10 batches: [time:0.0m 25.893124103546143s, Accuracy: 75.0]
After 15 batches: [time:0.0m 37.31352925300598s, Accuracy: 75.0]
After 20 batches: [time:0.0m 48.05142378807068s, Accuracy: 72.61904761904762]
After 25 batches: [time:0.0m 59.50429081916809s, Accuracy: 69.71153846153845]
After 30 batches: [time:1.0m 11.316333770751953s, Accuracy: 68.95161290322581]
After 35 batches: [time:1.0m 22.33365750312805s, Accuracy: 68.40277777777779]
Test accuracy: 67.66666666666666
Test time: 1.0m 25.707111358642578s


# 3. Model Evaluation

### 3.1. Accuracy

In [ ]:
print(f'Accuracy for the model is {test_acc*100} %')

### 3.2 Precision

In [ ]:
print(f'Precision for the model is {test_precision*100} %')

### 3.3 Recall

In [ ]:
print(f'Recall for the model is {test_recall*100} %')

### 3.4 F1 Score

In [ ]:
print(f'F1 score for the model is {test_f1*100} %')